# [1045. Customers Who Bought All Products](https://leetcode.com/problems/customers-who-bought-all-products/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Customer

<pre>+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| customer_id | int     |
| product_key | int     |
+-------------+---------+</pre>
This table may contain duplicates rows. 
customer_id is not NULL.
product_key is a foreign key (reference column) to Product table.
 

Table: Product

<pre>+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| product_key | int     |
+-------------+---------+</pre>
product_key is the primary key (column with unique values) for this table.
 

Write a solution to report the customer ids from the Customer table that bought all the products in the Product table.

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Customer table:
<pre>+-------------+-------------+
| customer_id | product_key |
+-------------+-------------+
| 1           | 5           |
| 2           | 6           |
| 3           | 5           |
| 3           | 6           |
| 1           | 6           |
+-------------+-------------+</pre>
Product table:
<pre>+-------------+
| product_key |
+-------------+
| 5           |
| 6           |
+-------------+</pre>
Output: 
<pre>+-------------+
| customer_id |
+-------------+
| 1           |
| 3           |
+-------------+</pre>
Explanation: 
The customers who bought all the products (5 and 6) are customers with IDs 1 and 3.

In [1]:
# pandas schema

import pandas as pd

data = [[1, 5], [2, 6], [3, 5], [3, 6], [1, 6]]
customer = pd.DataFrame(data, columns=['customer_id', 'product_key']).astype(
    {'customer_id': 'Int64', 'product_key': 'Int64'})
data = [[5], [6]]
product = pd.DataFrame(data, columns=['product_key']).astype({'product_key': 'Int64'})

In [2]:
#converting to spark dataframe
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession.builder.getOrCreate()

customer_df = spark.createDataFrame(customer)
customer_df.show()

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
23/11/11 18:43:29 WARN Utils: Your hostname, svchost resolves to a loopback address: 127.0.1.1; using 172.18.28.34 instead (on interface eth0)
23/11/11 18:43:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/11 18:43:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+-----------+-----------+
|customer_id|product_key|
+-----------+-----------+
|          1|          5|
|          2|          6|
|          3|          5|
|          3|          6|
|          1|          6|
+-----------+-----------+


In [3]:
product_df = spark.createDataFrame(product)
product_df.show()

+-----------+
|product_key|
+-----------+
|          5|
|          6|
+-----------+


In [4]:
# solving in spark dataframe

customer_df \
    .groupBy('customer_id') \
    .agg(F.array(F.collect_set('product_key')).alias('product_key')) \
    .join(product_df.agg(F.array(F.collect_set('product_key')).alias('product_key')),
          on=['product_key'],how='inner') \
    .select('customer_id') \
    .show()

+-----------+
|customer_id|
+-----------+
|          3|
|          1|
+-----------+


In [5]:
# solving in spark SQL

customer_df.createOrReplaceTempView('customer')
product_df.createOrReplaceTempView('product')

spark.sql('''
select customer_id 
from (
    select
        customer_id,
        collect_set(product_key) as product_key 
    from customer
    group by customer_id
    ) c
inner join (
    select collect_set(product_key) as product_key from product
    ) p on
c.product_key = p.product_key;
''').show()

+-----------+
|customer_id|
+-----------+
|          3|
|          1|
+-----------+


In [6]:
# mysql equivalent:
'''
select customer_id 
from (
    select
        customer_id,
        group_concat(distinct product_key order by product_key) as product_key 
    from customer
    group by customer_id
    ) c
inner join (
    select group_concat(distinct product_key order by product_key) as product_key from product
    ) p on
c.product_key = p.product_key;
'''

'\nselect customer_id \nfrom (\n    select\n        customer_id,\n        group_concat(distinct product_key order by product_key) as product_key \n    from customer\n    group by customer_id\n    ) c\ninner join (\n    select group_concat(distinct product_key order by product_key) as product_key from product\n    ) p on\nc.product_key = p.product_key;\n'

In [7]:
spark.stop()